# TNT Sticky Rouse: Flow Curve Fitting

## Objectives
- Fit TNT Sticky Rouse model to flow curve data
- Analyze sticker-dominated vs Rouse-dominated regimes
- Visualize effective relaxation times and mode contributions
- Demonstrate Bayesian inference for associative polymer systems

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTStickyRouse

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_sticky_rouse_param_names,
    plot_sticky_rouse_effective_times,
    plot_mode_decomposition,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

print("Setup complete. JAX devices:", jax.devices())

## Theory: Sticky Rouse Model

The Sticky Rouse model describes polymers with transient physical crosslinks (stickers):

**Key Physics:**
- Rouse chain has N modes with relaxation times τ_R,k
- Physical crosslinks (stickers) have lifetime τ_s
- Effective relaxation time: **τ_eff,k = max(τ_R,k, τ_s)**

**Regime Analysis:**
- When τ_s > τ_R,k: Sticker dominates → all modes relax on τ_s (sticker plateau)
- When τ_R,k > τ_s: Rouse dynamics dominate → mode-specific relaxation

**Steady-State Viscosity:**
$$\sigma(\dot{\gamma}) = \sum_k \frac{G_k \tau_{\textrm{eff},k} \dot{\gamma}}{1 + (\tau_{\textrm{eff},k} \dot{\gamma})^2} + \eta_s \dot{\gamma}$$

**Parameters (n_modes=3):**
- G_0, τ_R,0: First mode modulus and Rouse time
- G_1, τ_R,1: Second mode
- G_2, τ_R,2: Third mode
- τ_s: Sticker lifetime (critical parameter)
- η_s: Solvent viscosity

## Load Data

In [ ]:
# Load ARES flow curve data
gamma_dot, stress = load_ml_ikh_flow_curve("ARES_up")

print(f"Data shape: {len(gamma_dot)} points")
print(f"Shear rate range: {gamma_dot.min():.2e} - {gamma_dot.max():.2e} s^-1")
print(f"Stress range: {stress.min():.2e} - {stress.max():.2e} Pa")

# Plot raw data
fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(gamma_dot, stress, 'ko', label='ARES data', markersize=8)
ax.set_xlabel('Shear Rate (s$^{-1}$)', fontsize=12)
ax.set_ylabel('Stress (Pa)', fontsize=12)
ax.set_title('Flow Curve: Carbopol Gel', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.close("all")
plt.close('all')

print("\n⚠️  Note: Carbopol is not an ionomer. This demonstrates the Sticky Rouse workflow and sticker-mode analysis.")

## NLSQ Fitting

In [ ]:
# Initialize model with 3 modes
model = TNTStickyRouse(n_modes=3)
param_names = get_tnt_sticky_rouse_param_names(n_modes=3)
print(f"Model parameters ({len(param_names)}): {param_names}")

# Fit using NLSQ
print("\nFitting with NLSQ...")
start_time = time.time()
model.fit(gamma_dot, stress, test_mode="flow_curve", method='scipy')
fit_time = time.time() - start_time

# Compute metrics
stress_pred_train = model.predict(gamma_dot, test_mode="flow_curve")
metrics_nlsq = compute_fit_quality(stress, stress_pred_train)

print(f"\nFit completed in {fit_time:.2f} seconds")
print(f"R² = {metrics_nlsq['R2']:.6f}")
print(f"RMSE = {metrics_nlsq['RMSE']:.4e} Pa")

## Fitted Parameters

In [ ]:
# Extract fitted parameters
params_nlsq = {name: model.parameters.get_value(name) for name in param_names}

print("\nFitted Parameters:")
print("-" * 50)
for name, value in params_nlsq.items():
    if 'tau' in name:
        print(f"{name:10s} = {value:12.4e} s")
    elif 'eta' in name:
        print(f"{name:10s} = {value:12.4e} Pa·s")
    else:
        print(f"{name:10s} = {value:12.4e} Pa")

# Compute effective relaxation times
tau_s = params_nlsq['tau_s']
print(f"\nSticker lifetime: τ_s = {tau_s:.4e} s")
print("\nEffective Relaxation Times:")
print("-" * 50)
for i in range(3):
    tau_R = params_nlsq[f'tau_R_{i}']
    tau_eff = max(tau_R, tau_s)
    regime = "STICKER-DOMINATED" if tau_s > tau_R else "ROUSE-DOMINATED"
    print(f"Mode {i}: τ_R = {tau_R:.4e} s, τ_eff = {tau_eff:.4e} s ({regime})")

## Physical Analysis: Effective Relaxation Times

In [ ]:
gamma_dot_fine = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
stress_pred = model.predict(gamma_dot_fine, test_mode='flow_curve')
# Visualize sticker-mode interaction
fig = plot_sticky_rouse_effective_times(model)
plt.close("all")
plt.close('all')

## NLSQ Prediction vs Data

In [ ]:
# Compute metrics for plot title
metrics = compute_fit_quality(stress, model.predict(gamma_dot, test_mode='flow_curve'))

# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R² = {metrics["R2"]:.4f})'
)
plt.close("all")

## Mode Decomposition

In [ ]:
# Plot individual mode contributions
fig = plot_mode_decomposition(model, gamma_dot_fine, "flow_curve")
plt.close("all")
plt.close('all')

## Sticker-Limited Plateau Analysis

In [ ]:
# Analyze sticker plateau regime
tau_s = params_nlsq['tau_s']
sticker_shear_rate = 1.0 / tau_s
print(f"Sticker-limited plateau region: γ̇ ~ 1/τ_s = {sticker_shear_rate:.4e} s^-1")

# Count modes in sticker-dominated regime
n_sticker_modes = sum(1 for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)
print(f"\nNumber of sticker-dominated modes: {n_sticker_modes}/3")

# Plateau modulus estimate (sum of sticker-dominated modes)
G_plateau = sum(params_nlsq[f'G_{i}'] for i in range(3) if params_nlsq[f'tau_R_{i}'] < tau_s)
print(f"Sticker plateau modulus: G_plateau ≈ {G_plateau:.4e} Pa")

## Bayesian Inference

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    result_bayes = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    result_bayes = model.fit_bayesian(
        gamma_dot, stress,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


## Convergence Diagnostics

In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(result_bayes, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
# Compare point estimates
print_parameter_comparison(model, result_bayes.posterior_samples, param_names)

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(result_bayes, 'to_inference_data'):
    display_arviz_diagnostics(result_bayes, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

## Posterior Predictive Distribution

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(result_bayes, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot, stress, model, result_bayes,
        test_mode="flow_curve", param_names=param_names,
        log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

## Physical Interpretation

In [ ]:
# Get posterior samples
posterior = result_bayes.posterior_samples

# Extract posterior means
params_bayes = {name: float(np.mean(posterior[name])) for name in param_names}
tau_s_bayes = params_bayes['tau_s']

print("Physical Interpretation (Posterior Means):")
print("=" * 60)
print(f"\nSticker Lifetime: τ_s = {tau_s_bayes:.4e} s")
print(f"Critical Shear Rate: γ̇_c ~ 1/τ_s = {1.0/tau_s_bayes:.4e} s^-1")

print("\nMode-by-Mode Analysis:")
print("-" * 60)
for i in range(3):
    G_i = params_bayes[f'G_{i}']
    tau_R_i = params_bayes[f'tau_R_{i}']
    tau_eff_i = max(tau_R_i, tau_s_bayes)
    ratio = tau_s_bayes / tau_R_i
    
    print(f"\nMode {i}:")
    print(f"  G_{i} = {G_i:.4e} Pa")
    print(f"  τ_R,{i} = {tau_R_i:.4e} s")
    print(f"  τ_eff,{i} = {tau_eff_i:.4e} s")
    print(f"  τ_s/τ_R = {ratio:.2f}")
    
    if ratio > 1.0:
        print(f"  ✓ STICKER-DOMINATED (τ_s > τ_R): Mode contributes to sticker plateau")
    else:
        print(f"  ✓ ROUSE-DOMINATED (τ_R > τ_s): Mode exhibits intrinsic Rouse dynamics")

# Estimate zero-shear viscosity
eta_0 = sum(params_bayes[f'G_{i}'] * max(params_bayes[f'tau_R_{i}'], tau_s_bayes) for i in range(3)) + params_bayes['eta_s']
print(f"\nZero-Shear Viscosity: η_0 = {eta_0:.4e} Pa·s")

# Total modulus
G_total = sum(params_bayes[f'G_{i}'] for i in range(3))
print(f"Total Modulus: G_total = {G_total:.4e} Pa")

## Save Results

In [ ]:
# Save results to disk
output_path = save_tnt_results(model, result_bayes, "sticky_rouse", "flow_curve", param_names)
print(f"Results saved to: {output_path}")